# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [19]:
# imports
import os
from openai import OpenAI
from dotenv import load_dotenv
import requests
from IPython.display import Markdown, display

load_dotenv(override=True)

openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
openrouter_base_url = os.getenv('OPENROUTER_BASE_URL')

client = OpenAI(api_key=openrouter_api_key, base_url=openrouter_base_url)

In [12]:
# constants

MODEL_GPT = 'openai/gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_URL = "http://localhost:11434/api/generate"

In [ ]:
# set up environment

In [13]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [14]:
def explain(question: str, model: str = MODEL_GPT) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a senior software engineer. Explain clearly and technically."},
            {"role": "user", "content": question}
        ],
        temperature=0.3
    )
    
    return response.choices[0].message.content

In [15]:
def build_payload(question: str, model: str = MODEL_GPT) -> str:
    payload = {}
    if model == MODEL_GPT:
        payload = {
            "model": MODEL_GPT,
            "messages": [
                {"role": "system", "content": "You are a senior software engineer. Explain clearly and technically."},
                {"role": "user", "content": question}
            ],
            "temperature": 0.3
        }
    elif model == MODEL_LLAMA:
        payload = {
            "model": MODEL_LLAMA,
            "prompt": f"You are a senior engineer. Explain clearly:\n\n{question}",
            "stream": False
        }
    else:
        raise ValueError(f"Invalid model: {model}")

    return payload

In [20]:
def printmd(string):
    display(Markdown(string))

In [ ]:
# Get gpt-4o-mini to answer, without streaming
payload = build_payload(question)
response = client.chat.completions.create(**payload)

printmd(response.choices[0].message.content)


The provided code snippet is a Python expression that utilizes a generator function along with a set comprehension. Let's break it down step by step:

### Components of the Code

1. **`yield from`**: This is a keyword in Python used within a generator function to yield all values from an iterable. It effectively delegates part of its operations to another generator or iterable, allowing for a more concise and readable way to yield multiple values.

2. **Set Comprehension**: The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It creates a set of unique values based on the condition specified.

3. **`book.get("author")`**: This method attempts to retrieve the value associated with the key `"author"` from each dictionary `book` in the `books` iterable. If the key does not exist, it returns `None`.

4. **`if book.get("author")`**: This condition filters the books. It ensures that only those books that have a non-`None` value for the `"author"` key are included in the set comprehension.

### What the Code Does

- The code iterates over a collection called `books`, which is expected to be an iterable (like a list) of dictionaries where each dictionary represents a book.
- For each `book`, it attempts to get the value of the `"author"` key.
- If the `"author"` key exists and has a truthy value (i.e., not `None` or an empty string), that value is added to a set.
- The set comprehension ensures that only unique author names are collected, as sets inherently do not allow duplicate values.
- Finally, `yield from` yields each unique author name from the set one by one.

### Why Use This Code

1. **Efficiency**: By using a set comprehension, the code efficiently collects unique authors without needing to manually check for duplicates.
  
2. **Readability**: The use of `yield from` along with a set comprehension makes the code concise and easier to read, as it clearly indicates the intention of yielding unique authors.

3. **Lazy Evaluation**: Since this is a generator expression, it allows for lazy evaluation. This means that authors are generated one at a time and only when requested, which can be beneficial for memory usage when dealing with large datasets.

### Example

Suppose `books` is defined as follows:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},  # Duplicate author
    {"title": "Book 4"},  # No author
    {"title": "Book 5", "author": "Author C"},
]
```

The code would yield:

- "Author A"
- "Author B"
- "Author C"

In summary, the code efficiently collects and yields unique authors from a list of books, filtering out any entries without an author.

In [22]:
# Get Llama 3.2 to answer
payload = build_payload(question, MODEL_LLAMA)
response = requests.post(OLLAMA_URL, json=payload)
printmd(response.json()["response"])

This line of code uses a feature called generator expressions, which allows us to create iterators on the fly. Let's break it down:

1. `for book in books`: This part iterates over a collection (likely a list or dictionary) named `books`. It assigns each item in the collection to the variable `book`.

2. `if book.get("author")`: This is a conditional statement that filters out any items from the collection that do not have an "author" key.

3. `{...}`: This is a set literal, equivalent to a dictionary comprehension in some cases, but here it's used for a generator expression. It allows us to transform each item `book` into something else using the expression `{book.get("author")}`.

4. `yield from ...`: Here's where things get interesting. `yield from` is an expression that "yields" all items from another iterable, rather than just creating one.

So, putting it all together, this line of code does the following:

- It iterates over a collection of books.
- It filters out any books without an author.
- For each book with an author, it yields the author's name as if they were in a separate generator expression.

In terms of why this might be used, consider a situation where you're trying to retrieve the authors from multiple sources (like different APIs or databases), and you want to do so in a way that is efficient and easy to understand. Using `yield from` allows you to create one function that can handle these different sources, rather than having separate functions for each source.

For example:

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}
```

You could then call this function with your `books` collection and iterate over the resulting authors. This approach makes it easy to add support for new sources in the future, by just adding another `yield from` expression.

```python
def get_authors_from_api(api_url):
    # Logic to fetch authors from API goes here
    return [...author_list...]

books = [
    {"title": "Book 1", "author": "Author 1"},
    {"title": "Book 2", "author": None},
    ...
]
for author in get_authors(books):
    print(author)
```